In [1]:
import pandas as  pd

train = pd.read_csv("labeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
test = pd.read_csv("testData.tsv",header=0,delimiter="\t",quoting=3)
# print(unlabeled_train.shape)
# print(test.shape)
# print(train.shape)

In [85]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review,remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    words = review_text.lower().split()
    if remove_stopwords==True:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return words


In [86]:
import nltk.data
# 加载punkt tokenizer
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")

def review_to_sentences(review,tokenizer,remove_stopwords=False):
    # 返回一个list，分词；strip()去除首尾空格
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_to_wordlist(raw_sentence,remove_stopwords))
    # 返回值形状：[[第一个分句],[第二个分句],[第三个分句]...]->组成review
    return sentences

In [87]:
sentences = []
print("从训练集中解析句子")
for review in train["review"]:
    sentences += review_to_sentences(review,tokenizer)
    # 形状：[第一条review,第二条review]，其中每一条review对应:[[...],[...],[...]]见上

print("从未标记的训练集中解析句子")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review,tokenizer)

从训练集中解析句子


/var/folders/9c/fs4ghgt579d29fb6jsxdrls40000gn/T/ipykernel_4601/3485203933.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()
/var/folders/9c/fs4ghgt579d29fb6jsxdrls40000gn/T/ipykernel_4601/3485203933.py:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


从未标记的训练集中解析句子


In [88]:
num_features=300
min_words_count =40
downsampling=1e-3
context= 10
num_workers = 4

from gensim.models import word2vec
print("training model...")
model = word2vec.Word2Vec(sentences,workers=num_workers,vector_size=num_features,sample =downsampling,\
                          window=context,min_count = min_words_count)
model.init_sims(replace=True)
model_name = "300features_40minwords_10context"
model.save(model_name)


training model...


/var/folders/9c/fs4ghgt579d29fb6jsxdrls40000gn/T/ipykernel_4601/3181478921.py:11: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)
